## Collecting data 

### Importing packages 

In [1]:
import ccxt   # You need to install ccxt package to run the following code. Try : pip install ccxt
import time
import numpy as np
import pandas as pd

### Getting the coinpairs 

In [2]:
# List of the coins for which we have data on the transfer time

transfer_time_coins = ['BTC', 'ETH', 'LTC', 'XMR', 'DASH', 'ETC', 'ZEC', 'BTG', 'DGC', 'RDD', 'PPC', 'NVC', 'VTC', 
                       'NMC', 'FTC', 'BLK', 'AUR']  

# Function to get the common coinpairs at two different exchanges intersected with the coins for which we have data on the
# transfer time

def get_symbols(ex1, ex2, transfer_time_coins = transfer_time_coins):
    ex1_symbols = list(getattr(ccxt, ex1)().fetchTickers().keys())    # Coinpairs for the first exchange  e.g. [ETH/BTC, ADA/BTC]
    ex2_symbols = list(getattr(ccxt, ex2)().fetchTickers().keys())    # Coinpairs for the second exchange
    
    opportunity_coins = []
    for pair in (set(ex1_symbols) & set(ex2_symbols)):                # set(ex1_symbols) & set(ex2_symbols) : common coinpairs
        coin = pair[:pair.find('/')]                                  # The first coin in the coinpair  e.g [ETH, ADA]
        opportunity_coins.append(coin)                                   
        
    study_coins = set(opportunity_coins) & set(transfer_time_coins)   # List of the coins that appear in both exchanges and 
                                                                      # we have data for the transfer time  e.g [ETH]
    study_symbols = []
    for pair in (set(ex1_symbols) & set(ex2_symbols)):                # Adding the second coin  e.g [ETH/BTC]
        for coin in study_coins:
            if coin == pair[:pair.find('/')]:
                study_symbols.append(pair)
                
    return study_symbols

In [3]:
get_symbols('binance', 'kraken')

['XMR/BTC', 'LTC/BTC', 'ETC/ETH', 'ETH/BTC', 'DASH/BTC', 'ETC/BTC', 'ZEC/BTC']

### Getting the average ask price 

In [3]:
# Function to get the average ask price for a certain coin based on the trading amount

def avgcoinask(exchange, coinpair, btclimit):
    amountbtc = []
    quantity = []
    averageprice = 0
    lastprice = 0

    while True:  
        for i in getattr(ccxt, exchange)().fetch_order_book(coinpair).get('asks'):
            amountbtc.append(i[1] * i[0])
            quantity.append(i[1])
            if sum(amountbtc) > btclimit:
                lastprice = i[0]
                break
        break
                
    amountbtc[-1] = amountbtc[-1] - (sum(amountbtc)-btclimit)
    quantity[-1] = amountbtc[-1] / lastprice
    averageprice = sum(amountbtc)  /sum(quantity)
    return averageprice

In [21]:
avgcoinask(exchange = 'binance', coinpair = 'ETH/BTC', btclimit = 0.1)

0.027825985252227816

In [7]:
avgcoinask(exchange = 'binance', coinpair = 'ETH/BTC', btclimit = 1)

0.027829052791993117

In [22]:
avgcoinask(exchange = 'binance', coinpair = 'ETH/BTC', btclimit = 10)

0.027855130955874725

### Getting the average bid price 

In [4]:
# Function to get the average bid price for a certain coin based on the trading amount

def avgcoinbid(exchange, coinpair, btclimit):
    amountbtc = []
    quantity = []
    averageprice = 0
    lastprice = 0    
    
    while True:       
        for i in getattr(ccxt, exchange)().fetch_order_book(coinpair).get('bids'):
            amountbtc.append(i[1] * i[0])
            quantity.append(i[1])
            if sum(amountbtc) > btclimit:
                lastprice = i[0]
                break
        break
        
    amountbtc[-1] = amountbtc[-1] - (sum(amountbtc)-btclimit)
    quantity[-1] = amountbtc[-1] / lastprice
    averageprice = sum(amountbtc) / sum(quantity)
    return averageprice

In [26]:
avgcoinbid(exchange = 'kraken', coinpair = 'ETH/BTC', btclimit = 0.1)

0.02782

In [27]:
avgcoinbid(exchange = 'kraken', coinpair = 'ETH/BTC', btclimit = 1)

0.02778417735106473

In [28]:
avgcoinbid(exchange = 'kraken', coinpair = 'ETH/BTC', btclimit = 10)

0.027715297998988198

### Getting the difference between aks price at the first exchange and bid price at the second exchange 

In [5]:
def difference(ex1, ex2, coinpairs, btcamount):
    a = pd.DataFrame(index = [0])
    index1 = 0

    for pair in coinpairs:
        diffs = [[],[]]
        try:
            ask = avgcoinask(ex1, pair, btcamount)
            bid = avgcoinbid(ex2, pair, btcamount)
            diff = (bid - ask) / ((bid + ask) / 2)
            diffs[1].append(diff)
            diffs[0].append(time.time() - start_time)
            df = pd.DataFrame({"Difference " + pair: diffs[1], "Time " + pair: diffs[0]}, index= [index1])
            a = pd.concat([a,df], axis = 1)
        except Exception:
                pass

    return a    

In [6]:
start_time = time.time()
difference(ex1 = 'binance', ex2 = 'kraken', coinpairs = ['ETH/BTC', 'XMR/BTC'], btcamount = 0.1)

,Difference ETH/BTC,Time ETH/BTC,Difference XMR/BTC,Time XMR/BTC
0,-0.001687,2.598533,-0.000907,5.260754


### Running the difference function for a certain amount of time

In [10]:
def collectingdata(ex1, ex2, amount, duration_minutes, timesleep = 0):
    filename = '{0}_{1}_{2}_{3}'.format(ex1, ex2, amount, duration_minutes)
    coinpairs = get_symbols(ex1, ex2)
    a = difference(ex1, ex2, coinpairs, amount)
    
    while True:
        try:
            if (time.time() - start_time) < (duration_minutes * 60):
                c = a.append(difference(ex1, ex2, coinpairs, amount))
                a = c
                a.to_csv(filename)
                time.sleep(timesleep)
            else:
                return(a)
                break
        except Exception:
            pass

In [11]:
# This is how we produced the datasets
start_time = time.time()
diffs = [[],[]]
collectingdata('binance', 'kraken', 0.1, 1)

,Difference LTC/BTC,Time LTC/BTC,Difference DASH/BTC,Time DASH/BTC,Difference ZEC/BTC,Time ZEC/BTC,Difference ETC/ETH,Time ETC/ETH,Difference ETC/BTC,Time ETC/BTC,Difference ETH/BTC,Time ETH/BTC,Difference XMR/BTC,Time XMR/BTC
0,-0.002392,4.427927,-0.003537,7.397033,-0.002592,9.451360,-0.003207,11.598059,-0.003410,13.747417,-0.001883,15.795171,-0.000906,18.049273
0,-0.002267,20.507896,-0.003520,22.652406,-0.002855,24.648636,-0.003279,27.367374,-0.001706,29.725595,-0.001991,33.000205,-0.000906,35.050614
0,-0.003023,37.094765,-0.003586,39.658478,-0.003194,42.316361,-0.003279,44.467279,-0.002662,46.928341,-0.002350,49.078034,-0.000906,51.123055
0,-0.002897,53.377568,-0.003076,55.629974,-0.002231,57.884106,-0.001569,59.833381,-0.005111,61.975102,-0.001955,64.125773,-0.000777,66.489349
